In [1]:
import rioxarray as riox
from rasterio.enums import Resampling
import numpy as np
import rasterio as rio

In [2]:
file = 'composite_subset.tif'
output_file = '100m_mean_min_max_grid.tif'

raster = riox.open_rasterio(file)

# 100 m grid
grid_res = 100

downscale_factor = raster.rio.resolution()[0]/grid_res

# caluculate new height and width using downscale_factor
new_width = raster.rio.width * downscale_factor
new_height = raster.rio.height * downscale_factor
 
# resample raster to grid - 3 methods = mean, min, max
down_sampled_mean = raster.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.average)
down_sampled_min = raster.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.min)
down_sampled_max = raster.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.max)

In [3]:
mean_min_max_grid = np.vstack([down_sampled_mean, down_sampled_min, down_sampled_max])
print(mean_min_max_grid.shape)

(30, 666, 643)


In [6]:
with rio.open(file) as src:
    meta = src.meta
    transform = src.transform

transform = src.transform * src.transform.scale(
        (src.width / down_sampled_mean.shape[-1]),
        (src.height / down_sampled_mean.shape[-2])
)

out_meta = meta.copy()
out_meta.update({
    "count": mean_min_max_grid.shape[0],
    'height': mean_min_max_grid.shape[1],
    'width': mean_min_max_grid.shape[2],
    'transform': transform
})

# Save the merged TIFF
with rio.open(output_file, 'w', **out_meta) as dest:
    dest.write(mean_min_max_grid)